###Import necessary libraries

In [ ]:
! pip install optuna

In [ ]:
import sklearn
import xgboost

print(f"Scikit-learn version: {sklearn.__version__}")
print(f"XGBoost version: {xgboost.__version__}")


In [ ]:
!pip uninstall scikit-learn -y
!pip install scikit-learn==1.5.2

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import os,os.path
import re
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split, RandomizedSearchCV
import optuna
from sklearn.feature_selection import RFECV
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.utils import resample
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from scipy.stats import bootstrap
from sklearn.preprocessing import MinMaxScaler

###Load the datasets

In [ ]:
ibd = pd.read_excel('IBD biomarkers reduced.xlsx')
ibd

In [ ]:
gc = pd.read_excel('IBD biomarkers in GC.xlsx')
gc

In [ ]:
cc = pd.read_excel('IBD biomarkers in CC.xlsx')
cc

In [ ]:
features = ibd.drop(columns=['Group'])

# Initialize the Min-Max Scaler
scaler = MinMaxScaler()

# Apply Min-Max scaling to the selected features
scaled_features = scaler.fit_transform(features)

# Replace the original feature columns with the scaled ones
ibd = ibd.copy()
ibd.loc[:, features.columns] = scaled_features

# Display the first few rows of the scaled DataFrame
print(ibd.head())


In [ ]:
features = gc.drop(columns=['Group'])

# Initialize the Min-Max Scaler
scaler = MinMaxScaler()

# Apply Min-Max scaling to the selected features
scaled_features = scaler.fit_transform(features)

# Replace the original feature columns with the scaled ones
gc = gc.copy()
gc.loc[:, features.columns] = scaled_features

# Display the first few rows of the scaled DataFrame
print(gc.head())


In [ ]:
features = cc.drop(columns=['Group'])

# Initialize the Min-Max Scaler
scaler = MinMaxScaler()

# Apply Min-Max scaling to the selected features
scaled_features = scaler.fit_transform(features)

# Replace the original feature columns with the scaled ones
cc = cc.copy()
cc.loc[:, features.columns] = scaled_features

# Display the first few rows of the scaled DataFrame
print(cc.head())


#XGBoost

In [ ]:
X = ibd.drop(['Group'], axis=1)
y = ibd['Group']

# Encode categorical target labels into numerical labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Initialize XGBoost classifier
model = xgb.XGBClassifier(eval_metric='logloss', random_state=42)

# Train the model
model.fit(X_train, y_train)

# Make predictions on the original test set
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')

# Print evaluation metrics
print(f'Accuracy: {accuracy:.4f}')
print(f'Test Precision: {precision:.4f}')
print(f'Test F1 Score: {f1:.4f}')
print(f'Test Recall: {recall:.4f}')

##Random Search

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Define the parameter grid for RandomizedSearchCV
param_dist = {
    'n_estimators': np.arange(50, 200, 10),
    'max_depth': np.arange(3, 10),
    'learning_rate': np.linspace(0.01, 0.3, 10),
    'subsample': np.linspace(0.5, 1.0, 10),
    'colsample_bytree': np.linspace(0.5, 1.0, 10),
    'gamma': np.linspace(0, 0.5, 5),
    'min_child_weight': np.arange(1, 6)
}

# Initialize the XGBoost classifier
xgb = XGBClassifier(random_state=42)

# Set up RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=xgb, param_distributions=param_dist, n_iter=100,
    scoring='roc_auc', cv=5, verbose=1, random_state=42, n_jobs=-1
)

# Fit the RandomizedSearchCV model on the training data
random_search.fit(X_train, y_train)

# Get the best parameters from the random search
best_params = random_search.best_params_
print("Best Parameters:", best_params)

# Predict on the original test set
y_pred = random_search.predict(X_test)

# Calculate evaluation metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

# Print metrics
print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")

##Bayesian Optimization

In [ ]:
# Define the objective function for Optuna
def objective(trial):
    params = {
        'booster': trial.suggest_categorical('booster', ['gbtree', 'dart']),
        'objective': 'binary:logistic',
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'gamma': trial.suggest_float('gamma', 0, 0.5),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'min_child_weight': trial.suggest_float('min_child_weight', 0.5, 5),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'n_estimators': trial.suggest_int('n_estimators', 50, 300)
    }

    # Initialize the XGBoost model with the suggested hyperparameters
    model = xgb.XGBClassifier(**params)

    # Evaluate using cross-validation
    cv_scores = cross_val_score(model, X_train, y_train, cv=5, scoring='roc_auc')

    # Return the mean accuracy from cross-validation
    return cv_scores.mean()

# Create a study to maximize accuracy
study = optuna.create_study(direction='maximize')

# Enqueue the parameters obtained from previous RandomizedSearchCV results
study.enqueue_trial({
    'booster': 'gbtree',
    'objective': 'binary:logistic',
    'learning_rate': 0.3,
    'gamma':  0.25,
    'max_depth': 5,
    'min_child_weight': 1,
    'subsample':0.6111111111111112,
    'colsample_bytree':0.9444444444444444,
    'n_estimators': 180
})

# Optimize the study using 50 trials
study.optimize(objective, n_trials=50)

# Print the best parameters and cross-validation accuracy
print(f"Best Parameters: {study.best_params}")
print(f"Best Cross-validation Accuracy: {study.best_value:.4f}")

# Train the final model with the best parameters on the training data
best_params = study.best_params
final_model = xgb.XGBClassifier(**best_params)
final_model.fit(X_train, y_train)

# Make predictions on the original test set
y_pred = final_model.predict(X_test)

# Evaluate the final model on the test set
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')

# Print test set performance metrics
print(f"Test Accuracy: {accuracy:.4f}")
print(f"Test Precision: {precision:.4f}")
print(f"Test F1 Score: {f1:.4f}")
print(f"Test Recall: {recall:.4f}")

In [ ]:
selected_features = X_train.columns.tolist()

# Best parameters from Bayesian Optimization XGBoost
best_params_xg = {
    'learning_rate': 0.08704216750565849,
    'max_depth':  6,
    'n_estimators': 263,
    'gamma': 0.38647728030000605,
    'min_child_weight': 1.07625802095207,
    'subsample':  0.6847025379867031,
    'colsample_bytree':0.5264245161481004,
    'objective': 'binary:logistic',
    'booster': 'gbtree',
}
# Create the XGBoost classifier with the best parameters
final_model_xg = xgb.XGBClassifier(**best_params_xg)

# Perform cross-validation
cv_scores = cross_val_score(final_model_xg, X_train[selected_features], y_train, cv=5, scoring='roc_auc')
print("Cross-validation scores:")
print(cv_scores)
print(f"Mean CV accuracy: {np.mean(cv_scores):.4f}")

# Train the model
final_model_xg.fit(X_train[selected_features], y_train)

# Make predictions on the test set
y_pred_xg = final_model_xg.predict(X_test[selected_features])
y_pred_prob_xg = final_model_xg.predict_proba(X_test[selected_features])[:, 1]

# Calculate evaluation metrics
accuracy = accuracy_score(y_test, y_pred_xg)
precision = precision_score(y_test, y_pred_xg, average = 'weighted')
recall = recall_score(y_test, y_pred_xg, average = 'weighted')
f1 = f1_score(y_test, y_pred_xg, average = 'weighted')
roc_auc = roc_auc_score(y_test, y_pred_prob_xg)

# Calculate specificity
tn, fp, fn, tp = confusion_matrix(y_test, y_pred_xg).ravel()
specificity = tn / (tn + fp)

# Print the results
print(f'Test ROC AUC: {roc_auc:.2f}')
print(f'Test Accuracy: {accuracy:.2f}')
print(f'Test Precision: {precision:.2f}')
print(f'Test Recall: {recall:.2f}')
print(f'Test F1-Score: {f1:.2f}')
print(f'Test Specificity: {specificity:.2f}')


##95% CI

In [ ]:
n_iterations = 1000  # Number of bootstrap samples
metrics = {
    'accuracy': [],
    'precision': [],
    'recall': [],
    'f1': [],
    'roc_auc': [],
    'specificity': []
}

# Bootstrap resampling
for i in range(n_iterations):
    # Resample the test set with replacement
    X_test_resampled, y_test_resampled = resample(X_test, y_test, random_state=i)

    # Make predictions on the resampled test set
    y_pred_resampled = final_model_xg.predict(X_test_resampled)
    y_pred_prob_resampled = final_model_xg.predict_proba(X_test_resampled)[:, 1]

    # Compute confusion matrix for specificity
    tn, fp, fn, tp = confusion_matrix(y_test_resampled, y_pred_resampled).ravel()
    specificity_resampled = tn / (tn + fp)

    # Append each metric to the list
    metrics['accuracy'].append(accuracy_score(y_test_resampled, y_pred_resampled))
    metrics['precision'].append(precision_score(y_test_resampled, y_pred_resampled))
    metrics['recall'].append(recall_score(y_test_resampled, y_pred_resampled))
    metrics['f1'].append(f1_score(y_test_resampled, y_pred_resampled))
    metrics['roc_auc'].append(roc_auc_score(y_test_resampled, y_pred_prob_resampled))
    metrics['specificity'].append(specificity_resampled)

# Function to calculate 95% CI
def calc_ci(metric_list):
    lower = np.percentile(metric_list, 2.5)
    upper = np.percentile(metric_list, 97.5)
    return lower, upper

# Calculate 95% CIs for each metric
accuracy_ci = calc_ci(metrics['accuracy'])
precision_ci = calc_ci(metrics['precision'])
recall_ci = calc_ci(metrics['recall'])
f1_ci = calc_ci(metrics['f1'])
roc_auc_ci = calc_ci(metrics['roc_auc'])
specificity_ci = calc_ci(metrics['specificity'])

# Print the results with 95% CI
print(f"Test Accuracy: {accuracy:.2f} (95% CI: [{accuracy_ci[0]:.2f}, {accuracy_ci[1]:.2f}])")
print(f"Test Precision: {precision:.2f} (95% CI: [{precision_ci[0]:.2f}, {precision_ci[1]:.2f}])")
print(f"Test Recall: {recall:.2f} (95% CI: [{recall_ci[0]:.2f}, {recall_ci[1]:.2f}])")
print(f"Test F1-Score: {f1:.2f} (95% CI: [{f1_ci[0]:.2f}, {f1_ci[1]:.2f}])")
print(f"Test ROC AUC: {roc_auc:.2f} (95% CI: [{roc_auc_ci[0]:.2f}, {roc_auc_ci[1]:.2f}])")
print(f"Test Specificity: {specificity:.2f} (95% CI: [{specificity_ci[0]:.2f}, {specificity_ci[1]:.2f}])")


##Prediction on GC

In [ ]:
X_val_gc = gc.drop('Group', axis=1)
y_val_gc = gc['Group']

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y_val_gc)

# Assuming you have the feature names used in training saved
missing_features = [feature for feature in selected_features if feature not in X_val_gc.columns]

# Add the missing features to the validation set with zero values
missing_df = pd.DataFrame(0.0, index=X_val_gc.index, columns=missing_features)
X_val_gc = pd.concat([X_val_gc, missing_df], axis=1)

# Ensure the columns are in the same order as the training features
X_val_gc = X_val_gc[selected_features]

# Make predictions on the validation set
y_pred_prob_xg_gc = final_model_xg.predict_proba(X_val_gc)[:, 1]
y_pred_xg_gc = final_model_xg.predict(X_val_gc)

# calculate evaluation metrics
roc_auc_val = roc_auc_score(y_encoded, y_pred_prob_xg_gc)
accuracy_val = accuracy_score(y_encoded, y_pred_xg_gc)
precision_val = precision_score(y_encoded, y_pred_xg_gc, average="weighted", zero_division=1)
recall_val = recall_score(y_encoded, y_pred_xg_gc, average="weighted")
f1_val = f1_score(y_encoded, y_pred_xg_gc, average="weighted")
tn_val_cal, fp_val_cal, fn_val_cal, tp_val_cal = confusion_matrix(y_encoded, y_pred_xg_gc).ravel()
specificity_val = tn_val_cal / (tn_val_cal + fp_val_cal)

# Print evaluation metrics
print(f'Validation ROC AUC: {roc_auc_val:.2f}')
print(f'Validation Accuracy: {accuracy_val:.2f}')
print(f'Validation Precision: {precision_val:.2f}')
print(f'Validation Recall: {recall_val:.2f}')
print(f'Validation F1-Score: {f1_val:.2f}')
print(f'Validation Specificity: {specificity_val:.2f}')



##95% CI

In [ ]:
# Function to calculate 95% confidence interval using bootstrapping
def calculate_ci(metric_function, y_true, y_pred, y_pred_prob=None, n_bootstraps=1000, alpha=0.05):
    bootstrapped_scores = []
    for i in range(n_bootstraps):
        # Sample with replacement
        indices = np.random.randint(0, len(y_true), len(y_true))
        y_true_sample = y_true[indices]
        y_pred_sample = y_pred[indices]
        if y_pred_prob is not None:
            y_pred_prob_sample = y_pred_prob[indices]
            score = metric_function(y_true_sample, y_pred_prob_sample)
        else:
            score = metric_function(y_true_sample, y_pred_sample)
        bootstrapped_scores.append(score)
    # Calculate CI
    lower_bound = np.percentile(bootstrapped_scores, alpha / 2 * 100)
    upper_bound = np.percentile(bootstrapped_scores, (1 - alpha / 2) * 100)
    return lower_bound, upper_bound

# Calculate 95% CI for each metric
roc_auc_ci = calculate_ci(roc_auc_score, y_encoded, y_pred_prob_xg_gc)
accuracy_ci = calculate_ci(accuracy_score, y_encoded, y_pred_xg_gc)
precision_ci = calculate_ci(precision_score, y_encoded, y_pred_xg_gc)
recall_ci = calculate_ci(recall_score, y_encoded, y_pred_xg_gc)
f1_ci = calculate_ci(f1_score, y_encoded, y_pred_xg_gc)

# Specificity calculation with CI
def specificity(y_true, y_pred):
    tn, fp, _, _ = confusion_matrix(y_true, y_pred).ravel()
    return tn / (tn + fp)

specificity_ci = calculate_ci(specificity, y_encoded, y_pred_xg_gc)

# Print evaluation metrics with 95% CI
print(f'Validation ROC AUC: {roc_auc_val:.2f} (95% CI: {roc_auc_ci[0]:.2f} - {roc_auc_ci[1]:.2f})')
print(f'Validation Accuracy: {accuracy_val:.2f} (95% CI: {accuracy_ci[0]:.2f} - {accuracy_ci[1]:.2f})')
print(f'Validation Precision: {precision_val:.2f} (95% CI: {precision_ci[0]:.2f} - {precision_ci[1]:.2f})')
print(f'Validation Recall: {recall_val:.2f} (95% CI: {recall_ci[0]:.2f} - {recall_ci[1]:.2f})')
print(f'Validation F1-Score: {f1_val:.2f} (95% CI: {f1_ci[0]:.2f} - {f1_ci[1]:.2f})')
print(f'Validation Specificity: {specificity_val:.2f} (95% CI: {specificity_ci[0]:.2f} - {specificity_ci[1]:.2f})')


##Prediction on CRC

In [ ]:
X_val_cc = cc.drop('Group', axis=1)
y_val_cc = cc['Group']

# Encode the target variable
label_encoder = LabelEncoder()
y_encoded_cc = label_encoder.fit_transform(y_val_cc)

# Assuming you have the feature names used in training saved
missing_features = [feature for feature in selected_features if feature not in X_val_cc.columns]

# Add the missing features to the validation set with zero values
missing_df = pd.DataFrame(0.0, index=X_val_cc.index, columns=missing_features)
X_val_cc = pd.concat([X_val_cc, missing_df], axis=1)

# Ensure the columns are in the same order as the training features
X_val_cc = X_val_cc[selected_features]
# Get the predicted probabilities
y_pred_prob_xg_cc = final_model_xg.predict_proba(X_val_cc)[:, 1]

# Predict the class labels
y_pred_xg_cc = final_model_xg.predict(X_val_cc)

# Calculate evaluation metrics
roc_auc_val_xg = roc_auc_score(y_encoded_cc, y_pred_prob_xg_cc)

# Calculate accuracy, precision, recall, f1-score
accuracy_val_xg = accuracy_score(y_encoded_cc, y_pred_xg_cc)
precision_val_xg = precision_score(y_encoded_cc, y_pred_xg_cc, average="weighted", zero_division=1)
recall_val_xg = recall_score(y_encoded_cc, y_pred_xg_cc, average="weighted")
f1_val_xg = f1_score(y_encoded_cc, y_pred_xg_cc, average="weighted")

# Calculate confusion matrix
conf_matrix = confusion_matrix(y_encoded_cc, y_pred_xg_cc)

# Ensure confusion matrix has the correct dimensions for TN, FP, FN, TP
if conf_matrix.shape == (2, 2):
    tn_val_xg, fp_val_xg, fn_val_xg, tp_val_xg = conf_matrix.ravel()
    specificity_val_xg = tn_val_xg / (tn_val_xg + fp_val_xg)
else:
    specificity_val_xg = 'N/A'

# Print evaluation metrics
print(f'Validation ROC AUC: {roc_auc_val_xg:.2f}')
print(f'Validation Accuracy: {accuracy_val_xg:.2f}')
print(f'Validation Precision: {precision_val_xg:.2f}')
print(f'Validation Recall: {recall_val_xg:.2f}')
print(f'Validation F1-Score: {f1_val_xg:.2f}')
print(f'Validation Specificity: {specificity_val_xg:.2f}')


##95% CI

In [ ]:
def calculate_metrics(y_true, y_pred, y_pred_prob):
    metrics = {}
    metrics['roc_auc'] = roc_auc_score(y_true, y_pred_prob)
    metrics['accuracy'] = accuracy_score(y_true, y_pred)
    metrics['precision'] = precision_score(y_true, y_pred, average="weighted", zero_division=1)
    metrics['recall'] = recall_score(y_true, y_pred, average="weighted")
    metrics['f1'] = f1_score(y_true, y_pred, average="weighted")

    # Confusion matrix and specificity
    conf_matrix = confusion_matrix(y_true, y_pred)
    if conf_matrix.shape == (2, 2):
        tn, fp, fn, tp = conf_matrix.ravel()
        metrics['specificity'] = tn / (tn + fp)
    else:
        metrics['specificity'] = 'N/A'

    return metrics

# Bootstrap function to compute 95% confidence intervals
def bootstrap_ci(X, y, model, n_iterations=1000, alpha=0.95):
    metrics_list = []
    for i in range(n_iterations):
        # Resample the dataset
        X_resampled, y_resampled = resample(X, y, random_state=i)

        # Predict the probabilities and labels
        y_pred_prob = model.predict_proba(X_resampled)[:, 1]
        y_pred = model.predict(X_resampled)

        # Calculate metrics and store them
        metrics = calculate_metrics(y_resampled, y_pred, y_pred_prob)
        metrics_list.append(metrics)

    # Calculate the confidence intervals
    ci_dict = {}
    for metric in metrics_list[0].keys():
        values = [m[metric] for m in metrics_list if m[metric] != 'N/A']
        lower_bound = np.percentile(values, (1 - alpha) / 2 * 100)
        upper_bound = np.percentile(values, (1 + alpha) / 2 * 100)
        ci_dict[metric] = (lower_bound, upper_bound)

    return ci_dict

# Compute 95% confidence intervals using bootstrap
ci = bootstrap_ci(X_val_cc, y_encoded_cc, xg_model, n_iterations=1000, alpha=0.95)

# Print 95% confidence intervals for the metrics
print("\n95% Confidence Intervals:")
for metric, bounds in ci.items():
    print(f'{metric.capitalize()} CI: {bounds[0]:.2f} - {bounds[1]:.2f}')


#Random Forest

In [ ]:
X = ibd.drop(['Group'], axis=1)
y = ibd['Group']

# Encode categorical target labels into numerical labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Create the Random Forest classifier
clf = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the classifier on the training data
clf.fit(X_train, y_train)

# Make predictions on the test set
y_pred = clf.predict(X_test)

# Calculate accuracy on the test set
test_accuracy = accuracy_score(y_test, y_pred)
print(f"Test Accuracy: {test_accuracy:.2f}")

# Calculate F1 score on the test set
test_f1 = f1_score(y_test, y_pred, average='weighted')
print(f"Test F1 Score: {test_f1:.2f}")

# Calculate precision on the test set
test_precision = precision_score(y_test, y_pred, average='weighted')
print(f"Test Precision: {test_precision:.2f}")

# Calculate recall on the test set
test_recall = recall_score(y_test, y_pred, average='weighted')
print(f"Test Recall: {test_recall:.2f}")



##Random Search

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Define the parameter distribution for RandomizedSearchCV
param_dist = {
    'rf__n_estimators': [int(x) for x in np.linspace(start=200, stop=2000, num=10)],
    'rf__max_features': ['sqrt', 'log2'],
    'rf__max_depth': [int(x) for x in np.linspace(10, 300, num=20)] + [None],
    'rf__min_samples_split': [2, 5, 10, 15],
    'rf__min_samples_leaf': [1, 2, 4, 6],
    'rf__bootstrap': [True, False]
}

# Initialize the pipeline: Random Forest
pipeline = Pipeline([
    ('rf', RandomForestClassifier(random_state=42))
])

# Initialize RandomizedSearchCV
rf_random = RandomizedSearchCV(estimator=pipeline, param_distributions=param_dist,
                               n_iter=100, cv=StratifiedKFold(5), verbose=2,
                               random_state=42, n_jobs=-1, scoring='roc_auc')

# Fit RandomizedSearchCV to the original training data
rf_random.fit(X_train, y_train)

# Print the best parameters found by RandomizedSearchCV
print("Best parameters found by RandomizedSearchCV:")
print(rf_random.best_params_)

# Predict on the original test data
y_pred = rf_random.best_estimator_.predict(X_test)
y_prob = rf_random.best_estimator_.predict_proba(X_test)[:, 1]

# Evaluate the model with default threshold
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')
roc_auc = roc_auc_score(y_test, y_prob)

# Print evaluation metrics
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"ROC AUC: {roc_auc:.4f}")


##Bayesian Optimization

In [ ]:
# Define the best parameters found from RandomizedSearchCV RF
best_params_random = {
    'n_estimators': 600,
    'min_samples_split': 5,
    'min_samples_leaf':2,
    'max_features': 'sqrt',
    'max_depth': 132,
    'bootstrap': True,
    'class_weight': 'balanced'
}

def objective(trial):
    n_estimators = trial.suggest_int('n_estimators', max(100, best_params_random['n_estimators'] - 200), best_params_random['n_estimators'] + 200)
    min_samples_split = trial.suggest_int('min_samples_split', max(2, best_params_random['min_samples_split'] - 3), best_params_random['min_samples_split'] + 3)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', max(1, best_params_random['min_samples_leaf'] - 2), best_params_random['min_samples_leaf'] + 2)
    max_features = trial.suggest_categorical('max_features', ['sqrt', 'log2'])
    max_depth = trial.suggest_categorical('max_depth', [None, 10, 55, 132, 300])
    bootstrap = trial.suggest_categorical('bootstrap', [True, False])
    class_weight = trial.suggest_categorical('class_weight', ['balanced', None])
    criterion = trial.suggest_categorical('criterion', ['gini', 'entropy'])
    min_impurity_decrease = trial.suggest_float('min_impurity_decrease', 0.0, 0.01)

    # Initialize RandomForestClassifier with suggested hyperparameters
    clf = RandomForestClassifier(
        n_estimators=n_estimators,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        max_features=max_features,
        max_depth=max_depth,
        bootstrap=bootstrap,
        class_weight=class_weight,
        criterion=criterion,
        min_impurity_decrease=min_impurity_decrease,
        random_state=42
    )

    # Use cross-validation to evaluate the classifier
    cv_scores = cross_val_score(clf, X_train, y_train, cv=5, scoring='roc_auc')
    return np.mean(cv_scores)

# Perform optimization with Optuna
study = optuna.create_study(direction='maximize')

# Enqueue the trial with the best parameters from RandomizedSearchCV
study.enqueue_trial(best_params_random)
study.optimize(objective, n_trials=50)

# Print the best parameters and best score from Optuna
print("Best Parameters from Optuna:", study.best_params)

# Retrieve the best model and evaluate on the test set
best_params_optuna = study.best_params
best_clf = RandomForestClassifier(**best_params_optuna, random_state=50)
best_clf.fit(X_train, y_train)
y_pred = best_clf.predict(X_test)
y_prob = best_clf.predict_proba(X_test)[:, 1]

# Evaluate on test set
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')
roc_auc = roc_auc_score(y_test, y_prob)

print(f"Test Accuracy: {accuracy:.4f}")
print(f"Test Precision: {precision:.4f}")
print(f"Test Recall: {recall:.4f}")
print(f"Test F1 Score: {f1:.4f}")
print(f"Test ROC AUC: {roc_auc:.4f}")


In [ ]:
selected_features_rf = X_train.columns.tolist()

# Best parameters from Bayesian Optimization RF
best_params_rf = {
    'n_estimators': 421,
    'min_samples_split': 11,
    'min_samples_leaf': 1,
    'max_features': 'sqrt',
    'bootstrap': False,
    'criterion': 'gini',
    'max_depth': 55,
    'min_impurity_decrease': 0.0012231815602346612
}

## Create the Random Forest classifier with the best parameters
final_model_rf = RandomForestClassifier(**best_params_rf, random_state=50)

# Perform cross-validation with 5 folds
cv_scores = cross_val_score(final_model_rf, X_train[selected_features_rf], y_train, cv=5, scoring='roc_auc')
print("Cross-validation scores:")
print(cv_scores)
print(f"Mean CV accuracy: {np.mean(cv_scores):.4f}")

# Train the final model on the entire training data
final_model_rf.fit(X_train[selected_features_rf], y_train)

# Make predictions on the test set
y_pred_rf = final_model_rf.predict(X_test[selected_features_rf])
y_pred_prob_rf = final_model_rf.predict_proba(X_test[selected_features_rf])[:, 1]

# Calculate evaluation metrics on the test set
accuracy = accuracy_score(y_test, y_pred_rf)
precision = precision_score(y_test, y_pred_rf, average = 'weighted')
recall = recall_score(y_test, y_pred_rf, average = 'weighted')
f1 = f1_score(y_test, y_pred_rf, average = 'weighted')
auc_roc = roc_auc_score(y_test, y_pred_prob_rf)

tn, fp, fn, tp = confusion_matrix(y_test, y_pred_rf).ravel()
specificity = tn / (tn + fp)

# Print the results for test data
print(f'Test Accuracy: {accuracy:.2f}')
print(f'Test Precision: {precision:.2f}')
print(f'Test Recall: {recall:.2f}')
print(f'Test F1-Score: {f1:.2f}')
print(f'Test AUC-ROC: {auc_roc:.2f}')
print(f'Test Specificity: {specificity:.2f}')


##95% CI

In [ ]:
# Bootstrap resampling for 95% CI calculation
n_iterations = 1000  # Number of bootstrap iterations
metrics = {
    'accuracy': [],
    'precision': [],
    'recall': [],
    'f1': [],
    'auc_roc': [],
    'specificity': []
}

for i in range(n_iterations):
    # Resample the test set with replacement
    X_test_resampled, y_test_resampled = resample(X_test, y_test, random_state=i)

    # Make predictions on the resampled test set
    y_pred_resampled = final_model_rf.predict(X_test_resampled[selected_features_rf])
    y_pred_prob_resampled = final_model_rf.predict_proba(X_test_resampled[selected_features_rf])[:, 1]

    # Compute confusion matrix for specificity
    tn, fp, fn, tp = confusion_matrix(y_test_resampled, y_pred_resampled).ravel()
    specificity_resampled = tn / (tn + fp)

    # Append each metric to the list
    metrics['accuracy'].append(accuracy_score(y_test_resampled, y_pred_resampled))
    metrics['precision'].append(precision_score(y_test_resampled, y_pred_resampled, average='weighted'))
    metrics['recall'].append(recall_score(y_test_resampled, y_pred_resampled, average='weighted'))
    metrics['f1'].append(f1_score(y_test_resampled, y_pred_resampled, average='weighted'))
    metrics['auc_roc'].append(roc_auc_score(y_test_resampled, y_pred_prob_resampled))
    metrics['specificity'].append(specificity_resampled)

# Function to calculate 95% CI
def calc_ci(metric_list):
    lower = np.percentile(metric_list, 2.5)
    upper = np.percentile(metric_list, 97.5)
    return lower, upper

# Calculate 95% CIs for each metric
accuracy_ci = calc_ci(metrics['accuracy'])
precision_ci = calc_ci(metrics['precision'])
recall_ci = calc_ci(metrics['recall'])
f1_ci = calc_ci(metrics['f1'])
auc_roc_ci = calc_ci(metrics['auc_roc'])
specificity_ci = calc_ci(metrics['specificity'])

# Final test metrics
y_pred_rf = final_model_rf.predict(X_test[selected_features_rf])
y_pred_prob_rf = final_model_rf.predict_proba(X_test[selected_features_rf])[:, 1]
accuracy = accuracy_score(y_test, y_pred_rf)
precision = precision_score(y_test, y_pred_rf, average='weighted')
recall = recall_score(y_test, y_pred_rf, average='weighted')
f1 = f1_score(y_test, y_pred_rf, average='weighted')
auc_roc = roc_auc_score(y_test, y_pred_prob_rf)
tn, fp, fn, tp = confusion_matrix(y_test, y_pred_rf).ravel()
specificity = tn / (tn + fp)

# Print results with 95% CI
print(f'Test Accuracy: {accuracy:.2f} (95% CI: [{accuracy_ci[0]:.2f}, {accuracy_ci[1]:.2f}])')
print(f'Test Precision: {precision:.2f} (95% CI: [{precision_ci[0]:.2f}, {precision_ci[1]:.2f}])')
print(f'Test Recall: {recall:.2f} (95% CI: [{recall_ci[0]:.2f}, {recall_ci[1]:.2f}])')
print(f'Test F1-Score: {f1:.2f} (95% CI: [{f1_ci[0]:.2f}, {f1_ci[1]:.2f}])')
print(f'Test AUC-ROC: {auc_roc:.2f} (95% CI: [{auc_roc_ci[0]:.2f}, {auc_roc_ci[1]:.2f}])')
print(f'Test Specificity: {specificity:.2f} (95% CI: [{specificity_ci[0]:.2f}, {specificity_ci[1]:.2f}])')


##Prediction on GC

In [ ]:
# Validation set
X_val_gc_rf = gc.drop('Group', axis=1)
y_val_gc_rf = gc['Group']

# Encode categorical target labels into numerical labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y_val_gc_rf)

# Identify the features that are missing in the validation set
missing_features = [feature for feature in selected_features_rf if feature not in X_val_gc_rf.columns]

# Add the missing features to the validation set with zero values using pd.concat
missing_df_rf = pd.DataFrame(0, index=X_val_gc_rf.index, columns=missing_features)
X_val_gc_rf = pd.concat([X_val_gc_rf, missing_df_rf], axis=1)

# Ensure the columns are in the same order as the training features
X_val_gc_rf = X_val_gc_rf[selected_features_rf]

# Make predictions on the validation set without converting to NumPy array
y_pred_prob_rf_gc = final_model_rf.predict_proba(X_val_gc_rf)[:, 1]
y_pred_rf_gc = final_model_rf.predict(X_val_gc_rf)

# Calculate evaluation metrics
accuracy_val = accuracy_score(y_encoded, y_pred_rf_gc)
precision_val = precision_score(y_encoded, y_pred_rf_gc, average='weighted', zero_division=1)
recall_val = recall_score(y_encoded, y_pred_rf_gc, average='weighted')
f1_val = f1_score(y_encoded, y_pred_rf_gc, average='weighted')
roc_auc_val = roc_auc_score(y_encoded, y_pred_prob_rf_gc)

# Calculate specificity
tn_val, fp_val, fn_val, tp_val = confusion_matrix(y_encoded, y_pred_rf_gc).ravel()
specificity_val = tn_val / (tn_val + fp_val)

# Print evaluation metrics
print(f'Validation ROC AUC: {roc_auc_val:.2f}')
print(f'Validation Accuracy: {accuracy_val:.2f}')
print(f'Validation Precision: {precision_val:.2f}')
print(f'Validation Recall: {recall_val:.2f}')
print(f'Validation F1-Score: {f1_val:.2f}')
print(f'Validation Specificity: {specificity_val:.2f}')



##95% CI

In [ ]:
# Function to compute metrics
def compute_metrics(y_true, y_pred, y_pred_proba):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='weighted', zero_division=1)
    recall = recall_score(y_true, y_pred, average='weighted')
    f1 = f1_score(y_true, y_pred, average='weighted')
    roc_auc = roc_auc_score(y_true, y_pred_proba)

    # Confusion matrix to compute specificity
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    specificity = tn / (tn + fp)

    return accuracy, precision, recall, f1, roc_auc, specificity

# Number of bootstrap iterations
n_iterations = 1000
n_size = len(X_val_gc_rf)

# Initialize lists to store metric values for each bootstrap sample
accuracy_scores = []
precision_scores = []
recall_scores = []
f1_scores = []
roc_auc_scores = []
specificity_scores = []

# Bootstrap procedure
for i in range(n_iterations):
    # Resample the validation set with replacement
    X_resample, y_resample = resample(X_val_gc_rf, y_encoded, n_samples=n_size, random_state=i)

    # Make predictions on the resampled data
    y_pred_resample = final_model_rf.predict(X_resample)
    y_pred_proba_resample = final_model_rf.predict_proba(X_resample)[:, 1]

    # Calculate metrics for this bootstrap sample
    accuracy, precision, recall, f1, roc_auc, specificity = compute_metrics(y_resample, y_pred_resample, y_pred_proba_resample)

    # Store the metrics
    accuracy_scores.append(accuracy)
    precision_scores.append(precision)
    recall_scores.append(recall)
    f1_scores.append(f1)
    roc_auc_scores.append(roc_auc)
    specificity_scores.append(specificity)

# Calculate 95% confidence intervals for each metric
def calculate_confidence_interval(scores):
    lower_bound = np.percentile(scores, 2.5)
    upper_bound = np.percentile(scores, 97.5)
    return lower_bound, upper_bound

# Calculate and print 95% confidence intervals
accuracy_ci = calculate_confidence_interval(accuracy_scores)
precision_ci = calculate_confidence_interval(precision_scores)
recall_ci = calculate_confidence_interval(recall_scores)
f1_ci = calculate_confidence_interval(f1_scores)
roc_auc_ci = calculate_confidence_interval(roc_auc_scores)
specificity_ci = calculate_confidence_interval(specificity_scores)

print(f'Validation ROC AUC: {roc_auc_val:.2f}, 95% CI: [{roc_auc_ci[0]:.2f}, {roc_auc_ci[1]:.2f}]')
print(f'Validation Accuracy: {accuracy_val:.2f}, 95% CI: [{accuracy_ci[0]:.2f}, {accuracy_ci[1]:.2f}]')
print(f'Validation Precision: {precision_val:.2f}, 95% CI: [{precision_ci[0]:.2f}, {precision_ci[1]:.2f}]')
print(f'Validation Recall: {recall_val:.2f}, 95% CI: [{recall_ci[0]:.2f}, {recall_ci[1]:.2f}]')
print(f'Validation F1-Score: {f1_val:.2f}, 95% CI: [{f1_ci[0]:.2f}, {f1_ci[1]:.2f}]')
print(f'Validation Specificity: {specificity_val:.2f}, 95% CI: [{specificity_ci[0]:.2f}, {specificity_ci[1]:.2f}]')


##Predictions on CRC

In [ ]:
X_val_cc_rf = cc.drop('Group', axis=1)
y_val_cc_rf = cc['Group']

# Encode categorical target labels into numerical labels
label_encoder = LabelEncoder()
y_encoded_cc_rf = label_encoder.fit_transform(y_val_cc_rf)

# Identify the features that are missing in the validation set
missing_features = [feature for feature in selected_features_rf if feature not in X_val_cc_rf.columns]

# Add the missing features to the validation set with zero values using pd.concat
missing_df_rf = pd.DataFrame(0, index=X_val_cc_rf.index, columns=missing_features)
X_val_cc_rf = pd.concat([X_val_cc_rf, missing_df_rf], axis=1)

# Ensure the columns are in the same order as the training features
X_val_cc_rf = X_val_cc_rf[selected_features_rf]


rf_model.fit(X_val_cc_rf, y_encoded_cc_rf)
y_pred_prob_rf_cc = rf_model.predict_proba(X_val_cc_rf)[:, 1]

# Predict the class labels
y_pred_rf_cc = rf_model.predict(X_val_cc_rf)

# Calculate evaluation metrics
roc_auc_val_rf = roc_auc_score(y_encoded_cc_rf, y_pred_prob_rf_cc)

# Calculate accuracy, precision, recall, f1-score
accuracy_val_rf = accuracy_score(y_encoded_cc_rf, y_pred_rf_cc)
precision_val_rf = precision_score(y_encoded_cc_rf, y_pred_rf_cc, average="weighted", zero_division=1)
recall_val_rf = recall_score(y_encoded_cc_rf, y_pred_rf_cc, average="weighted")
f1_val_rf = f1_score(y_encoded_cc_rf, y_pred_rf_cc, average="weighted")

# Calculate confusion matrix
conf_matrix = confusion_matrix(y_encoded_cc_rf, y_pred_rf_cc)

# Ensure confusion matrix has the correct dimensions for TN, FP, FN, TP
if conf_matrix.shape == (2, 2):
    tn_val_rf, fp_val_rf, fn_val_rf, tp_val_rf = conf_matrix.ravel()
    specificity_val_rf = tn_val_rf / (tn_val_rf + fp_val_rf)
else:
    specificity_val_rf = 'N/A'

# Print evaluation metrics
print(f'Validation ROC AUC: {roc_auc_val_rf:.2f}')
print(f'Validation Accuracy: {accuracy_val_rf:.2f}')
print(f'Validation Precision: {precision_val_rf:.2f}')
print(f'Validation Recall: {recall_val_rf:.2f}')
print(f'Validation F1-Score: {f1_val_rf:.2f}')
print(f'Validation Specificity: {specificity_val_rf}')


##95% CI

In [ ]:
# Function to calculate specificity
def calculate_specificity(y_true, y_pred):
    conf_matrix = confusion_matrix(y_true, y_pred)
    if conf_matrix.shape == (2, 2):
        tn, fp, fn, tp = conf_matrix.ravel()
        return tn / (tn + fp) if (tn + fp) > 0 else 0
    return None

# Number of bootstrap samples
n_bootstraps = 1000
rng = np.random.default_rng(seed=42)

# Containers for bootstrap results
roc_auc_scores = []
accuracy_scores = []
precision_scores = []
recall_scores = []
f1_scores = []
specificity_scores = []

# Bootstrap process
for _ in range(n_bootstraps):
    # Resample with replacement
    indices = rng.choice(len(y_encoded_cc_rf), size=len(y_encoded_cc_rf), replace=True)
    y_true_resampled = y_encoded_cc_rf[indices]
    X_resampled = X_val_cc_rf.iloc[indices]

    # Predictions on resampled data
    y_pred_resampled = rf_model.predict(X_resampled)
    y_pred_prob_resampled = rf_model.predict_proba(X_resampled)[:, 1]

    # Compute metrics
    roc_auc_scores.append(roc_auc_score(y_true_resampled, y_pred_prob_resampled))
    accuracy_scores.append(accuracy_score(y_true_resampled, y_pred_resampled))
    precision_scores.append(precision_score(y_true_resampled, y_pred_resampled, average="weighted", zero_division=1))
    recall_scores.append(recall_score(y_true_resampled, y_pred_resampled, average="weighted"))
    f1_scores.append(f1_score(y_true_resampled, y_pred_resampled, average="weighted"))
    specificity_scores.append(calculate_specificity(y_true_resampled, y_pred_resampled))

# Calculate 95% CI for each metric
def get_confidence_interval(data):
    lower = np.percentile(data, 2.5)
    upper = np.percentile(data, 97.5)
    return lower, upper

roc_auc_ci = get_confidence_interval(roc_auc_scores)
accuracy_ci = get_confidence_interval(accuracy_scores)
precision_ci = get_confidence_interval(precision_scores)
recall_ci = get_confidence_interval(recall_scores)
f1_ci = get_confidence_interval(f1_scores)
specificity_ci = get_confidence_interval(specificity_scores)

# Print metrics with 95% CI
print(f'Validation ROC AUC: {roc_auc_val_rf:.2f} (95% CI: {roc_auc_ci[0]:.2f}-{roc_auc_ci[1]:.2f})')
print(f'Validation Accuracy: {accuracy_val_rf:.2f} (95% CI: {accuracy_ci[0]:.2f}-{accuracy_ci[1]:.2f})')
print(f'Validation Precision: {precision_val_rf:.2f} (95% CI: {precision_ci[0]:.2f}-{precision_ci[1]:.2f})')
print(f'Validation Recall: {recall_val_rf:.2f} (95% CI: {recall_ci[0]:.2f}-{recall_ci[1]:.2f})')
print(f'Validation F1-Score: {f1_val_rf:.2f} (95% CI: {f1_ci[0]:.2f}-{f1_ci[1]:.2f})')
print(f'Validation Specificity: {specificity_val_rf:.2f} (95% CI: {specificity_ci[0]:.2f}-{specificity_ci[1]:.2f})')

#LASSO


In [ ]:
X = ibd.drop(['Group'], axis=1)
y = ibd['Group']

# Encode categorical target labels into numerical labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Create Logistic Regression classifier with L1 regularization
log_reg = LogisticRegression(penalty='l1', solver='liblinear', random_state=42)

# Train the classifier
log_reg.fit(X_train, y_train)

# Predict on the test set
y_pred = log_reg.predict(X_test)

# Calculate evaluation metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

# Print the evaluation metrics
print(f'Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1 Score: {f1:.4f}')

##Random Search

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Define the parameter grid for Randomized Search
param_dist = {
    'C': np.logspace(-4, 4, 20),
    'solver': ['liblinear'],
    'max_iter': [1000, 5000, 10000, 20000],
    'tol': [1e-4, 1e-3, 1e-2, 1e-1]  }

# Create Logistic Regression classifier with L1 regularization (Lasso)
log_reg = LogisticRegression(penalty='l1', random_state=42)

# Set up the Randomized Search with cross-validation
random_search = RandomizedSearchCV(
    log_reg, param_distributions=param_dist, n_iter=100,
    scoring='roc_auc', cv=5, verbose=1, random_state=42, n_jobs=-1
)

# Fit the Randomized Search model
random_search.fit(X_train, y_train)

# Get the best parameters
best_params = random_search.best_params_
print("Best Parameters:", best_params)

# Predict on the test set with the best model
y_pred = random_search.predict(X_test)

# Calculate evaluation metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

# Print the evaluation metrics
print(f'Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1 Score: {f1:.4f}')


##Bayesian Optimization

In [ ]:
random_search_params = {'tol': 0.01, 'solver': 'liblinear', 'max_iter': 10000, 'C': 4.281332398719396}

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

def objective(trial):
    C = trial.suggest_float('C', 1e-4, 1e6, log=True)
    max_iter = trial.suggest_int('max_iter', 5000, 100000)
    tol = trial.suggest_float('tol', 1e-4, 1e-2, log=True)
    solver = trial.suggest_categorical('solver', ['liblinear', 'saga'])
    clf = LogisticRegression(
        penalty='l1', C=C, max_iter=max_iter, tol=tol, solver=solver, random_state=42
    )

    score = cross_val_score(clf, X_train_balanced, y_train_balanced, cv=5, scoring='roc_auc').mean()
    return score

# Create the study
study = optuna.create_study(direction='maximize')

# Define the initial random search trial
def random_search_trial(trial):
    trial.suggest_float('C', 206.913808111479, 206.913808111479, log=True)
    trial.suggest_int('max_iter', 5000, 5000)
    trial.suggest_float('tol', 0.0001, 0.0001, log=True)
    trial.suggest_categorical('solver', ['liblinear'])

# Start the optimization
study.enqueue_trial(random_search_params)
study.optimize(objective, n_trials=100)

# Print the best parameters and best AUC-ROC score from Optuna
print("Best Parameters from Optuna:", study.best_params)
print("Best AUC-ROC Score from Optuna:", study.best_value)

# Retrieve the best parameters and train the model on the balanced data
best_params = study.best_params
clf = LogisticRegression(
    penalty='l1', **best_params, random_state=42
)
clf.fit(X_train_balanced, y_train_balanced)

# Predict on the test set
y_pred_prob = clf.predict_proba(X_test)[:, 1]
y_pred = clf.predict(X_test)

# Evaluate the model on the test set
roc_auc = roc_auc_score(y_test, y_pred_prob)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

# Print evaluation metrics
print(f"Test AUC-ROC: {roc_auc:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")
print(f"Test Precision: {precision:.4f}")
print(f"Test Recall: {recall:.4f}")
print(f"Test F1 Score: {f1:.4f}")


In [ ]:
selected_features_lasso = X_train.columns.tolist()

# Best parameters from Bayesian Optimization with Optuna
best_params_lasso = {
    'C': 0.9837039781542123,
    'max_iter':40222,
    'solver': 'saga',
    'tol': 0.000778168973127006
}

# Train the Logistic Regression model again using only the selected features
final_model_lasso = LogisticRegression(penalty='l1', **best_params_lasso, random_state=42)


# Perform cross-validation with 5 folds
cv_scores = cross_val_score(final_model_lasso, X_train[selected_features_lasso], y_train, cv=5, scoring='roc_auc')
print("Cross-validation scores:")
print(cv_scores)
print(f"Mean CV accuracy: {np.mean(cv_scores):.4f}")

final_model_lasso.fit(X_train[selected_features_lasso], y_train)

# Make predictions on the test set (predicted probabilities)
y_pred_proba_lasso = final_model_lasso.predict_proba(X_test[selected_features_lasso])[:, 1]
# Convert probabilities to predicted class labels
y_pred_lasso = final_model_lasso.predict(X_test[selected_features_lasso])

# Calculate evaluation metrics
accuracy = accuracy_score(y_test, y_pred_lasso)
precision = precision_score(y_test, y_pred_lasso, average = 'weighted')
recall = recall_score(y_test, y_pred_lasso, average = 'weighted')
f1 = f1_score(y_test, y_pred_lasso, average = 'weighted')
roc_auc = roc_auc_score(y_test, y_pred_proba_lasso)

# Calculate confusion matrix and extract TN, FP, FN, TP
tn, fp, fn, tp = confusion_matrix(y_test, y_pred_lasso).ravel()

# Calculate specificity
specificity = tn / (tn + fp)

print(f'Test ROC AUC: {roc_auc:.2f}')
print(f'Test Accuracy: {accuracy:.2f}')
print(f'Test Precision: {precision:.2f}')
print(f'Test Recall: {recall:.2f}')
print(f'Test F1-Score: {f1:.2f}')
print(f'Test Specificity: {specificity:.2f}')



##95% CI

In [ ]:
def calculate_metrics(y_true, y_pred, y_proba):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    metrics = {
        'accuracy': accuracy_score(y_true, y_pred),
        'precision': precision_score(y_true, y_pred, average='weighted', zero_division=1),
        'recall': recall_score(y_true, y_pred, average='weighted'),
        'f1': f1_score(y_true, y_pred, average='weighted'),
        'roc_auc': roc_auc_score(y_true, y_proba),
        'specificity': tn / (tn + fp) if (tn + fp) > 0 else 0
    }
    return metrics

# Initialize variables
n_bootstraps = 1000
metrics_list = []

# Bootstrapping
for _ in range(n_bootstraps):
    X_resampled, y_resampled = resample(
        X_test[selected_features_lasso], y_test, random_state=np.random.randint(0, 10000)
    )
    y_pred_resampled = final_model_lasso.predict(X_resampled)
    y_proba_resampled = final_model_lasso.predict_proba(X_resampled)[:, 1]
    metrics = calculate_metrics(y_resampled, y_pred_resampled, y_proba_resampled)
    metrics_list.append(metrics)

# Convert list of metrics to DataFrame
metrics_df = pd.DataFrame(metrics_list)

# Calculate 95% CI for each metric
ci_results = {}
for metric in metrics_df.columns:
    lower = np.percentile(metrics_df[metric], 2.5)
    upper = np.percentile(metrics_df[metric], 97.5)
    ci_results[metric] = (lower, upper)

print(f'Test ROC AUC: {metrics_df["roc_auc"].mean():.2f} (95% CI: {ci_results["roc_auc"][0]:.2f}-{ci_results["roc_auc"][1]:.2f})')
print(f'Test Accuracy: {metrics_df["accuracy"].mean():.2f} (95% CI: {ci_results["accuracy"][0]:.2f}-{ci_results["accuracy"][1]:.2f})')
print(f'Test Precision: {metrics_df["precision"].mean():.2f} (95% CI: {ci_results["precision"][0]:.2f}-{ci_results["precision"][1]:.2f})')
print(f'Test Recall: {metrics_df["recall"].mean():.2f} (95% CI: {ci_results["recall"][0]:.2f}-{ci_results["recall"][1]:.2f})')
print(f'Test F1-Score: {metrics_df["f1"].mean():.2f} (95% CI: {ci_results["f1"][0]:.2f}-{ci_results["f1"][1]:.2f})')
print(f'Test Specificity: {metrics_df["specificity"].mean():.2f} (95% CI: {ci_results["specificity"][0]:.2f}-{ci_results["specificity"][1]:.2f})')


##Predictions on GC

In [ ]:
# Validation set
X_val_gc_lasso = gc.drop('Group', axis=1)
y_val_gc_lasso = gc['Group']

# Encode categorical target labels into numerical labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y_val_gc_lasso)

# Identify the features that are missing in the validation set
missing_features = [feature for feature in selected_features_lasso if feature not in X_val_gc_lasso.columns]

# Add the missing features to the validation set with zero values using pd.concat
missing_df_lasso = pd.DataFrame(0, index=X_val_gc_lasso.index, columns=missing_features)
X_val_gc_lasso = pd.concat([X_val_gc_lasso, missing_df_lasso], axis=1)

# Ensure the columns are in the same order as the training features
X_val_gc_lasso = X_val_gc_lasso[selected_features_lasso]


#Make predictions on the validation set
y_pred_prob_lasso_gc= final_model_lasso.predict_proba(X_val_gc_lasso)[:, 1]
y_pred_lasso_gc = final_model_lasso.predict(X_val_gc_lasso)


# Calculate evaluation metrics
accuracy_val = accuracy_score(y_encoded, y_pred_lasso_gc)
precision_val = precision_score(y_encoded, y_pred_lasso_gc, average='weighted')
recall_val = recall_score(y_encoded, y_pred_lasso_gc, average="weighted")
f1_val = f1_score(y_encoded, y_pred_lasso_gc,  average="weighted")
roc_auc_val = roc_auc_score(y_encoded, y_pred_prob_lasso_gc)

# Calculate specificity
tn_val, fp_val, fn_val, tp_val = confusion_matrix(y_encoded, y_pred_lasso_gc).ravel()
specificity_val = tn_val / (tn_val + fp_val)

# Print evaluation metrics
print(f'Validation ROC AUC: {roc_auc_val:.2f}')
print(f'Validation Accuracy: {accuracy_val:.2f}')
print(f'Validation Precision: {precision_val:.2f}')
print(f'Validation Recall: {recall_val:.2f}')
print(f'Validation F1-Score: {f1_val:.2f}')
print(f'Validation Specificity: {specificity_val:.2f}')

##95% CI

In [ ]:
# Function to compute metrics
def compute_metrics(y_true, y_pred, y_pred_proba):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='weighted')
    recall = recall_score(y_true, y_pred, average="weighted")
    f1 = f1_score(y_true, y_pred, average="weighted")
    roc_auc = roc_auc_score(y_true, y_pred_proba)

    # Confusion matrix to compute specificity
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    specificity = tn / (tn + fp)

    return accuracy, precision, recall, f1, roc_auc, specificity

# Number of bootstrap iterations
n_iterations = 1000
n_size = len(X_val_gc_lasso)

# Initialize lists to store metric values for each bootstrap sample
accuracy_scores = []
precision_scores = []
recall_scores = []
f1_scores = []
roc_auc_scores = []
specificity_scores = []

# Bootstrap procedure
for i in range(n_iterations):
    # Resample the validation set with replacement
    X_resample, y_resample = resample(X_val_gc_lasso, y_encoded, n_samples=n_size, random_state=i)

    # Make predictions on the resampled data
    y_pred_resample = final_model_lasso.predict(X_resample)
    y_pred_proba_resample = final_model_lasso.predict_proba(X_resample)[:, 1]

    # Calculate metrics for this bootstrap sample
    accuracy, precision, recall, f1, roc_auc, specificity = compute_metrics(y_resample, y_pred_resample, y_pred_proba_resample)

    # Store the metrics
    accuracy_scores.append(accuracy)
    precision_scores.append(precision)
    recall_scores.append(recall)
    f1_scores.append(f1)
    roc_auc_scores.append(roc_auc)
    specificity_scores.append(specificity)

# Calculate 95% confidence intervals for each metric
def calculate_confidence_interval(scores):
    lower_bound = np.percentile(scores, 2.5)
    upper_bound = np.percentile(scores, 97.5)
    return lower_bound, upper_bound

# Print 95% confidence intervals
accuracy_ci = calculate_confidence_interval(accuracy_scores)
precision_ci = calculate_confidence_interval(precision_scores)
recall_ci = calculate_confidence_interval(recall_scores)
f1_ci = calculate_confidence_interval(f1_scores)
roc_auc_ci = calculate_confidence_interval(roc_auc_scores)
specificity_ci = calculate_confidence_interval(specificity_scores)

print(f'Validation ROC AUC: {roc_auc_val:.2f}, 95% CI: [{roc_auc_ci[0]:.2f}, {roc_auc_ci[1]:.2f}]')
print(f'Validation Accuracy: {accuracy_val:.2f}, 95% CI: [{accuracy_ci[0]:.2f}, {accuracy_ci[1]:.2f}]')
print(f'Validation Precision: {precision_val:.2f}, 95% CI: [{precision_ci[0]:.2f}, {precision_ci[1]:.2f}]')
print(f'Validation Recall: {recall_val:.2f}, 95% CI: [{recall_ci[0]:.2f}, {recall_ci[1]:.2f}]')
print(f'Validation F1-Score: {f1_val:.2f}, 95% CI: [{f1_ci[0]:.2f}, {f1_ci[1]:.2f}]')
print(f'Validation Specificity: {specificity_val:.2f}, 95% CI: [{specificity_ci[0]:.2f}, {specificity_ci[1]:.2f}]')


##Predictions on CRC

In [ ]:
X_val_cc_lasso = cc.drop('Group', axis=1)
y_val_cc_lasso = cc['Group']

# Encode categorical target labels into numerical labels
label_encoder = LabelEncoder()
y_encoded_cc_lasso = label_encoder.fit_transform(y_val_cc_lasso)

# Identify the features that are missing in the validation set
missing_features = [feature for feature in selected_features_lasso if feature not in X_val_cc_lasso.columns]

# Add the missing features to the validation set with zero values using pd.concat
missing_df_lasso = pd.DataFrame(0, index=X_val_cc_lasso.index, columns=missing_features)
X_val_cc_lasso = pd.concat([X_val_cc_lasso, missing_df_lasso], axis=1)

# Ensure the columns are in the same order as the training features
X_val_cc_lasso = X_val_cc_lasso[selected_features_lasso]

lasso_model.fit(X_val_cc_lasso, y_encoded_cc_lasso)

# Get the predicted probabilities and predictions
y_pred_prob_lasso_cc = lasso_model.predict_proba(X_val_cc_lasso)[:, 1]
y_pred_lasso_cc = lasso_model.predict(X_val_cc_lasso)

# Calculate evaluation metrics
roc_auc_val_lasso = roc_auc_score(y_encoded_cc_lasso, y_pred_prob_lasso_cc)
accuracy_val_lasso = accuracy_score(y_encoded_cc_lasso, y_pred_lasso_cc)
precision_val_lasso = precision_score(y_encoded_cc_lasso, y_pred_lasso_cc, average="weighted", zero_division=1)
recall_val_lasso = recall_score(y_encoded_cc_lasso, y_pred_lasso_cc, average="weighted")
f1_val_lasso = f1_score(y_encoded_cc_lasso, y_pred_lasso_cc, average="weighted")

# Specificity calculation
raw_predictions = final_model_lasso.predict(X_val_cc_lasso)
conf_matrix = confusion_matrix(y_encoded_cc_lasso, raw_predictions)
if conf_matrix.shape == (2, 2):
    tn, fp, fn, tp = conf_matrix.ravel()
    specificity_val_lasso = tn / (tn + fp)
else:
    specificity_val_lasso = 'N/A'

# Print evaluation metrics
print(f'Validation ROC AUC: {roc_auc_val_lasso:.2f}')
print(f'Validation Accuracy: {accuracy_val_lasso:.2f}')
print(f'Validation Precision: {precision_val_lasso:.2f}')
print(f'Validation Recall: {recall_val_lasso:.2f}')
print(f'Validation F1-Score: {f1_val_lasso:.2f}')
print(f'Validation Specificity: {specificity_val_lasso:.2f}')



##95% CI

In [ ]:
# Bootstrapping for metrics
def bootstrap_metric(metric_func, y_true, y_pred, n_resamples=1000):
    results = []
    for _ in range(n_resamples):
        indices = resample(range(len(y_true)), replace=True)
        y_true_sample, y_pred_sample = y_true[indices], y_pred[indices]
        results.append(metric_func(y_true_sample, y_pred_sample))
    return np.mean(results), np.percentile(results, [2.5, 97.5])

# Get predictions and probabilities
y_pred_prob = lasso_model.predict_proba(X_val_cc_lasso)[:, 1]
y_pred = lasso_model.predict(X_val_cc_lasso)

# Metrics and confidence intervals
metrics = {
    "ROC AUC": roc_auc_score,
    "Accuracy": accuracy_score,
    "Precision": lambda y_true, y_pred: precision_score(y_true, y_pred, average="weighted", zero_division=1),
    "Recall": lambda y_true, y_pred: recall_score(y_true, y_pred, average="weighted"),
    "F1-Score": lambda y_true, y_pred: f1_score(y_true, y_pred, average="weighted"),
}

for name, metric_func in metrics.items():
    mean, ci = bootstrap_metric(metric_func, y_encoded_cc_lasso, y_pred_prob if name == "ROC AUC" else y_pred)
    print(f"{name}: {mean:.2f} (95% CI: {ci[0]:.2f}, {ci[1]:.2f})")
